In [6]:
%pip install torch torchvision torchaudio transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [7]:
import torch
import pandas as pd
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration

# 모델과 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')

# CSV 파일 로드
df = pd.read_csv('/Volumes/1TB/스인재/esgc team project/data/crawling_tuned_data.csv')

print(f"데이터 크기: {df.shape}")
print(f"컬럼명: {list(df.columns)}")
print("\n첫 번째 기사 제목:")
print(df['title'].iloc[0])
print("\n첫 번째 기사 본문 일부:")
print(df['text'].iloc[0][:200] + "...")

text = "과거를 떠올려보자. 방송을 보던 우리의 모습을. 독보적인 매체는 TV였다. 온 가족이 둘러앉아 TV를 봤다. 간혹 가족들끼리 뉴스와 드라마, 예능 프로그램을 둘러싸고 리모컨 쟁탈전이 벌어지기도  했다. 각자 선호하는 프로그램을 ‘본방’으로 보기 위한 싸움이었다. TV가 한 대인지 두 대인지 여부도 그래서 중요했다. 지금은 어떤가. ‘안방극장’이라는 말은 옛말이 됐다. TV가 없는 집도 많다. 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다. 각자의 방에서 각자의 휴대폰으로, 노트북으로, 태블릿으로 콘텐츠 를 즐긴다."

raw_input_ids = tokenizer.encode(text)
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]

summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.


데이터 크기: (10952, 3)
컬럼명: ['original_url', 'title', 'text']

첫 번째 기사 제목:
이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”

첫 번째 기사 본문 일부:
이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포했습니다.   이 대통령은 재난 상황에서 부적절한 공직자들의 처신 문제도 거론했는데, 엄혹한 현장에서 음주가무를 즐기는 정신 나간 경우도 있었다며 엄히 단속하라고 주문했습니다.   보도에 이희연 기자입니다.   리포트  경기도 가평과 충남 서산 예산.....


'TV가 없는 집도 많아지고 미디어의 혜 택을 누릴 수 있는 방법은 늘어났다.'

In [8]:
def summarize_text(text):
    """
    주어진 텍스트를 요약하는 함수
    """
    if pd.isna(text) or len(text.strip()) == 0:
        return "텍스트가 없습니다."
    
    # 텍스트가 너무 길면 일부만 사용 (토큰 제한 때문)
    if len(text) > 2000:
        text = text[:2000]
    
    try:
        # 토큰화
        raw_input_ids = tokenizer.encode(text)
        input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
        
        # 요약 생성
        summary_ids = model.generate(torch.tensor([input_ids]), max_length=150, num_beams=4, early_stopping=True)
        summary = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)
        
        return summary
    except Exception as e:
        return f"요약 생성 중 오류 발생: {str(e)}"

# 첫 번째 기사 요약 테스트
first_article = df['title'].iloc[0]
print("원본 기사:")
print(first_article[:500] + "...")
print("\n요약:")
summary = summarize_text(first_article)
print(summary)

원본 기사:
이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”...

요약:
이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”
이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”


In [9]:
# 샘플 데이터로 여러 기사 요약 (처음 5개 기사)
sample_size = 5
sample_df = df.head(sample_size).copy()

print(f"{sample_size}개 기사 요약 중...")
summaries = []

for idx, row in sample_df.iterrows():
    print(f"기사 {idx + 1} 요약 중...")
    title = row['title']
    content = row['text']
    summary = summarize_text(content)
    
    summaries.append({
        '원본_제목': title,
        '요약': summary
    })

# 결과를 DataFrame으로 변환
summary_df = pd.DataFrame(summaries)

# 결과 출력
for i, row in summary_df.iterrows():
    print(f"\n{'='*50}")
    print(f"기사 {i+1}")
    print(f"제목: {row['원본_제목']}")
    print(f"요약: {row['요약']}")
    print(f"{'='*50}")

# 요약 결과를 CSV로 저장
summary_df.to_csv('/Volumes/1TB/스인재/esgc team project/news_summaries.csv', index=False, encoding='utf-8-sig')
print(f"\n요약 결과가 'news_summaries.csv' 파일로 저장되었습니다.")

5개 기사 요약 중...
기사 1 요약 중...
기사 2 요약 중...
기사 2 요약 중...
기사 3 요약 중...
기사 3 요약 중...
기사 4 요약 중...
기사 4 요약 중...
기사 5 요약 중...
기사 5 요약 중...

기사 1
제목: 이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”
요약: 이재명 대통령이 경기도 가평과 전남 담양, 경남 산청 등 6개 시군을 특별재난지역으로 선포하고 엄정한 단속을 주문했다.

기사 2
제목: 신경호 결심 공판 열려 검찰 “징역 3년 요구”
요약: 신경호 강원도교육감에 대한 결심공판이 어제22일 열렸는데   검찰은 징역형을 내려야한다고 주장했고   신경호 강원도교육감이 나타납니다.

기사 3
제목: ‘계엄 옹호’ 강준욱 자진 사퇴 최동석 ‘2차 가해’ 발언 등 논란 확산
요약: 강준욱 대통령실 국민통합비서관이 계엄 옹호 발언 등으로 결국 자진 사퇴했고, 최동석 인사혁신처장도 박원순 전 시장 사건을 두고 한 과거 발언이 2차 가해 논란에 섰다.

기사 4
제목: 이 대통령, 강선우 등 4명 청문보고서 재송부 요청
요약: 이재명 대통령은 국회에 강선우 여성가족부 장관 후보자를 포함해, 국방부와 국가보훈부, 통일부 장관 후보자에 대한 인사청문보고서를 내일까지 재송부해 달라고 요청했고  강유정 대통령실 대변인은 브리핑을 통해 이번주 내 임명을 마무리하고 신속한 국정 안정을 꾀하기 위해 기한은 오는 24일로 요청했다고 밝혔다.

기사 5
제목: 6곳 특별재난지역 선포 “공직자 부적절 행위 엄히 단속”
요약: 중앙재난안전대책본부는 경기 가평, 충남 서산 예산, 전남 담양, 경남 산청 합천을 집중호우 피해지역에 대한 사전조사를 토대로 대통령 재가를 받아 특별재난지역으로 우선 선포했다고 밝혔다.

요약 결과가 'news_summaries.csv' 파일로 저장되었습니다.

기사 1
제목: 이 대통령, 6개 시군 특별재난 지역 선포 “재난에 음주가무 엄히 단속”
요약: 이재명 대통령이 경기도 가평과 전남 

# 전체 데이터 요약 실행

테스트가 완료되었으니 이제 전체 데이터셋에 대해 요약을 진행합니다.
- GPU 사용 최적화
- 배치 처리로 메모리 효율성 향상
- 진행 상황 모니터링
- 에러 처리 및 복구

In [11]:
# GPU 확인 및 디바이스 설정
import time
import gc
import psutil

print("🔍 시스템 환경 확인")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")

# 디바이스 설정
if torch.cuda.is_available():
    device = torch.device('cuda')
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    print(f"🚀 GPU 사용: {gpu_name}")
    print(f"💾 GPU 메모리: {gpu_memory:.1f}GB")
elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
    device = torch.device('mps')
    print("🍎 Apple MPS GPU 사용")
else:
    device = torch.device('cpu')
    cpu_count = psutil.cpu_count()
    ram_gb = psutil.virtual_memory().total / 1024**3
    print(f"💻 CPU 사용: {cpu_count}코어")
    print(f"💾 RAM: {ram_gb:.1f}GB")

# 모델을 선택된 디바이스로 이동
print(f"\n📦 모델을 {device}로 이동 중...")
model = model.to(device)
print("✅ 모델 이동 완료")

# 메모리 사용량 확인
if device.type == 'cuda':
    allocated = torch.cuda.memory_allocated(device) / 1024**3
    print(f"💾 현재 GPU 메모리 사용량: {allocated:.2f}GB")

🔍 시스템 환경 확인
PyTorch 버전: 2.7.1
CUDA 사용 가능: False
🍎 Apple MPS GPU 사용

📦 모델을 mps로 이동 중...
✅ 모델 이동 완료
✅ 모델 이동 완료


In [16]:
# 전체 데이터 처리를 위한 최적화된 요약 함수
from tqdm import tqdm  # 진행 표시 라이브러리 import

def summarize_batch(texts, batch_size=8, max_length=150):
    """
    배치 단위로 텍스트 요약 처리 (GPU 최적화)
    """
    summaries = []
    failed_count = 0
    
    print(f"📊 총 {len(texts)}개 기사 처리 시작")
    print(f"⚙️ 배치 크기: {batch_size}")
    print(f"🎯 디바이스: {device}")
    
    start_time = time.time()
    
    for i in tqdm(range(0, len(texts), batch_size), desc="요약 진행"):
        batch_texts = texts[i:i+batch_size]
        batch_summaries = []
        
        for text in batch_texts:
            try:
                # 텍스트 전처리
                if pd.isna(text) or len(str(text).strip()) == 0:
                    batch_summaries.append("텍스트가 없습니다.")
                    failed_count += 1
                    continue
                
                # 텍스트 길이 제한
                text = str(text)
                if len(text) > 2000:
                    text = text[:2000]
                
                # 토큰화 및 디바이스 이동
                raw_input_ids = tokenizer.encode(text, max_length=1024, truncation=True)
                input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]
                input_tensor = torch.tensor([input_ids]).to(device)
                
                # 요약 생성
                with torch.no_grad():  # 메모리 절약
                    summary_ids = model.generate(
                        input_tensor,
                        max_length=max_length,
                        min_length=30,
                        num_beams=4,
                        early_stopping=True,
                        no_repeat_ngram_size=2,
                        length_penalty=1.0
                    )
                
                # 디코딩
                summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
                batch_summaries.append(summary)
                
            except Exception as e:
                error_msg = f"요약 생성 실패: {str(e)[:100]}"
                batch_summaries.append(error_msg)
                failed_count += 1
                print(f"⚠️ 기사 {i + len(batch_summaries)} 처리 중 오류: {e}")
        
        summaries.extend(batch_summaries)
        
        # 메모리 정리 (매 10배치마다)
        if i % (batch_size * 10) == 0:
            gc.collect()
            if device.type == 'cuda':
                torch.cuda.empty_cache()
                allocated = torch.cuda.memory_allocated(device) / 1024**3
                print(f"💾 메모리 정리 후 GPU 사용량: {allocated:.2f}GB")
        
        # 진행 상황 출력 (매 50배치마다)
        if (i // batch_size + 1) % 50 == 0:
            elapsed = time.time() - start_time
            processed = min(i + batch_size, len(texts))
            rate = processed / elapsed
            remaining = (len(texts) - processed) / rate / 60
            print(f"📈 진행: {processed}/{len(texts)} ({processed/len(texts)*100:.1f}%) - {rate:.1f}개/초 - 남은 시간: {remaining:.1f}분")
    
    end_time = time.time()
    total_time = end_time - start_time
    
    print(f"\n✅ 요약 완료!")
    print(f"⏱️ 총 처리 시간: {total_time/60:.1f}분")
    print(f"⚡ 평균 처리 속도: {len(texts)/total_time:.1f}개/초")
    print(f"❌ 실패한 기사: {failed_count}개")
    print(f"✅ 성공률: {(len(texts)-failed_count)/len(texts)*100:.1f}%")
    
    return summaries

print("🛠️ 배치 요약 함수 준비 완료 (tqdm 포함)")

🛠️ 배치 요약 함수 준비 완료 (tqdm 포함)


In [13]:
# 전체 데이터 확인 및 배치 크기 설정
print("📊 전체 데이터 분석")
print(f"총 기사 수: {len(df):,}개")

# 유효한 데이터만 필터링
valid_data = df.dropna(subset=['text']).copy()
valid_data = valid_data[valid_data['text'].str.len() > 50]  # 최소 50자 이상

print(f"유효한 기사: {len(valid_data):,}개")
print(f"제거된 기사: {len(df) - len(valid_data):,}개")

# 텍스트 길이 분석
text_lengths = valid_data['text'].str.len()
print(f"\\n📏 텍스트 길이 통계:")
print(f"평균: {text_lengths.mean():.0f}자")
print(f"중간값: {text_lengths.median():.0f}자")
print(f"최대: {text_lengths.max():,}자")
print(f"최소: {text_lengths.min():,}자")

# 디바이스별 배치 크기 설정
if device.type == 'cuda':
    gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1024**3
    if gpu_memory_gb >= 16:
        batch_size = 16
    elif gpu_memory_gb >= 8:
        batch_size = 12
    else:
        batch_size = 8
elif device.type == 'mps':
    batch_size = 6  # MPS는 보수적으로
else:
    batch_size = 4  # CPU는 작은 배치

print(f"\\n⚙️ 설정된 배치 크기: {batch_size}")

# 예상 처리 시간 계산
estimated_time = len(valid_data) / batch_size * 2  # 배치당 약 2초 가정
print(f"📅 예상 처리 시간: {estimated_time/60:.1f}분")

# 최종 확인
print(f"\\n🎯 처리 대상: {len(valid_data):,}개 기사")
print(f"🔧 사용 디바이스: {device}")
print(f"📦 배치 크기: {batch_size}")

# 데이터 준비
df_to_process = valid_data.copy()
print("\\n✅ 전체 데이터 처리 준비 완료")

📊 전체 데이터 분석
총 기사 수: 10,952개
유효한 기사: 10,812개
제거된 기사: 140개
\n📏 텍스트 길이 통계:
평균: 189자
중간값: 190자
최대: 200자
최소: 52자
\n⚙️ 설정된 배치 크기: 6
📅 예상 처리 시간: 60.1분
\n🎯 처리 대상: 10,812개 기사
🔧 사용 디바이스: mps
📦 배치 크기: 6
\n✅ 전체 데이터 처리 준비 완료


In [ ]:
# 전체 데이터 요약 실행
print("🚀 전체 데이터 요약 시작!")
print("="*60)

# 시작 전 메모리 상태 확인
if device.type == 'cuda':
    initial_memory = torch.cuda.memory_allocated(device) / 1024**3
    print(f"💾 시작 전 GPU 메모리: {initial_memory:.2f}GB")

# 요약 실행
try:
    all_summaries = summarize_batch(
        texts=df_to_process['text'].tolist(),
        batch_size=batch_size,
        max_length=150
    )
    
    print("\\n📊 요약 결과 생성 중...")
    
    # 결과 DataFrame 생성
    result_df = pd.DataFrame({
        '기사_번호': range(1, len(df_to_process) + 1),
        '원본_제목': df_to_process['title'].tolist(),
        '원본_내용': df_to_process['text'].tolist(),
        'KoBERT_요약': all_summaries,
        '원본_길이': df_to_process['text'].str.len(),
        '요약_길이': pd.Series(all_summaries).str.len(),
    })
    
    # 압축률 계산
    result_df['압축률(%)'] = (result_df['요약_길이'] / result_df['원본_길이'] * 100).round(1)
    
    # 요약 통계
    print(f"\\n📈 요약 통계:")
    print(f"총 처리된 기사: {len(result_df):,}개")
    print(f"평균 압축률: {result_df['압축률(%)'].mean():.1f}%")
    print(f"평균 요약 길이: {result_df['요약_길이'].mean():.0f}자")
    
    # 실패한 요약 확인
    failed_summaries = result_df[result_df['KoBERT_요약'].str.contains('실패|오류|없습니다')]
    print(f"실패한 요약: {len(failed_summaries)}개")
    
    print("\\n✅ 전체 데이터 요약 완료!")
    
except Exception as e:
    print(f"❌ 요약 처리 중 오류 발생: {e}")
    import traceback
    traceback.print_exc()

🚀 전체 데이터 요약 시작!
📊 총 10812개 기사 처리 시작
⚙️ 배치 크기: 6
🎯 디바이스: mps


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
요약 진행:   1%|          | 20/1802 [04:45<7:42:48, 15.58s/it]

In [ ]:
# 전체 요약 결과 저장
try:
    # 파일 경로 설정
    output_path = '/Volumes/1TB/스인재/esgc team project/kobert_full_news_summaries.csv'
    
    # CSV 파일로 저장
    result_df.to_csv(output_path, index=False, encoding='utf-8-sig')
    
    print(f"💾 결과 저장 완료!")
    print(f"📁 파일 위치: {output_path}")
    print(f"📊 저장된 데이터: {len(result_df):,}행 × {len(result_df.columns)}열")
    
    # 파일 크기 확인
    import os
    file_size_mb = os.path.getsize(output_path) / 1024 / 1024
    print(f"📦 파일 크기: {file_size_mb:.1f}MB")
    
    # 요약 품질 분석
    print(f"\\n📊 요약 품질 분석:")
    
    # 압축률 분포
    compression_ranges = [
        (0, 10, "매우 높은 압축"),
        (10, 20, "높은 압축"), 
        (20, 30, "적절한 압축"),
        (30, 50, "낮은 압축"),
        (50, 100, "매우 낮은 압축")
    ]
    
    for min_val, max_val, label in compression_ranges:
        count = len(result_df[(result_df['압축률(%)'] >= min_val) & (result_df['압축률(%)'] < max_val)])
        percentage = count / len(result_df) * 100
        print(f"  {label} ({min_val}-{max_val}%): {count:,}개 ({percentage:.1f}%)")
    
    print(f"\\n📈 전체 통계:")
    print(result_df[['원본_길이', '요약_길이', '압축률(%)']].describe())
    
except Exception as e:
    print(f"❌ 결과 저장 중 오류: {e}")

In [ ]:
# 요약 결과 샘플 확인
print("🔍 요약 결과 샘플 확인 (무작위 3개)")
print("="*80)

# 무작위로 3개 샘플 선택
import random
sample_indices = random.sample(range(len(result_df)), min(3, len(result_df)))

for i, idx in enumerate(sample_indices, 1):
    row = result_df.iloc[idx]
    print(f"\\n📰 샘플 {i} (기사 번호: {row['기사_번호']})")
    print(f"제목: {row['원본_제목'][:100]}...")
    print(f"원본 ({row['원본_길이']}자): {row['원본_내용'][:200]}...")
    print(f"요약 ({row['요약_길이']}자): {row['KoBERT_요약']}")
    print(f"압축률: {row['압축률(%)']}%")
    print("-" * 80)

# 최고/최저 압축률 기사 확인
print(f"\\n🏆 압축률 극값 사례:")
best_compression = result_df.loc[result_df['압축률(%)'].idxmin()]
worst_compression = result_df.loc[result_df['압축률(%)'].idxmax()]

print(f"\\n📉 최고 압축 (압축률: {best_compression['압축률(%)']}%):")
print(f"제목: {best_compression['원본_제목'][:100]}...")
print(f"요약: {best_compression['KoBERT_요약']}")

print(f"\\n📈 최저 압축 (압축률: {worst_compression['압축률(%)']}%):")
print(f"제목: {worst_compression['원본_제목'][:100]}...")
print(f"요약: {worst_compression['KoBERT_요약']}")

In [ ]:
# 메모리 정리 및 작업 완료
print("\\n🧹 메모리 정리 중...")

# 메모리 정리
gc.collect()

if device.type == 'cuda':
    torch.cuda.empty_cache()
    final_memory = torch.cuda.memory_allocated(device) / 1024**3
    max_memory = torch.cuda.max_memory_allocated(device) / 1024**3
    print(f"💾 최종 GPU 메모리 사용량: {final_memory:.2f}GB")
    print(f"💾 최대 GPU 메모리 사용량: {max_memory:.2f}GB")

# 최종 완료 메시지
print("\\n" + "="*60)
print("🎉 전체 뉴스 데이터 KoBERT 요약 완료!")
print("="*60)
print(f"✅ 처리된 기사 수: {len(result_df):,}개")
print(f"📁 결과 파일: kobert_full_news_summaries.csv")
print(f"📊 평균 압축률: {result_df['압축률(%)'].mean():.1f}%")
print(f"⏱️ 사용된 디바이스: {device}")
print("\\n다음 단계:")
print("1. 저장된 CSV 파일을 확인하세요")
print("2. 필요시 요약 품질 평가를 진행하세요")
print("3. 원본 데이터와 요약 데이터를 비교 분석하세요")
print("="*60)